In [ ]:
#Installing PyKeen
! pip install --upgrade pip
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git

In [ ]:
import torch
import pykeen
import pandas as pd
from pykeen import predict
from pykeen.datasets import Nations
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory
import numpy as np
from scipy.spatial.distance import euclidean
device = "cuda" if torch.cuda.is_available() else "cpu"


In [4]:
#Load model
model_name = "transE_on_query-result"

pykeen_model = torch.load("./"+ model_name+"/trained_model.pkl",map_location=torch.device('cpu'))

file_path = "./"+ model_name+'/training_triples/relation_to_id.tsv.gz'
df = pd.read_csv(file_path, sep='\t', compression='gzip', header=0)
rel_to_id = dict(zip(df.iloc[:, 1], df.iloc[:, 0]))
file_path = "./"+ model_name+'/training_triples/entity_to_id.tsv.gz'
df = pd.read_csv(file_path, sep='\t', compression='gzip', header=0)
ent_to_id = dict(zip(df.iloc[:, 1], df.iloc[:, 0]))

C:\Users\de137428\AppData\Local\Temp\ipykernel_25472\1118450277.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pykeen_model = torch.load("./"+ model_name+"/trained_mode

In [12]:
#Fetching embeddings from notation in example .ipynb
embeddings = pykeen_model.entity_representations[0](indices= None).detach()
chosen_paper_id = "<http://example.org/academic#Paper_10000166>"
paper_id = ent_to_id[chosen_paper_id]
embeddings_paper = embeddings[paper_id]

cites_relation = rel_to_id['<http://example.org/academic#cites>']
cites_relation_embedding = embeddings[cites_relation]
likely_cited_paper_embedding = embeddings_paper + cites_relation_embedding
print(likely_cited_paper_embedding)

tensor([ 3.4144e-03, -1.5768e-01,  1.9612e-01, -1.8815e-02,  1.5389e-01,
         4.9188e-04, -1.5908e-01, -1.7610e-01, -1.5934e-01, -1.7207e-01,
        -1.7405e-02, -1.7704e-01,  1.5766e-01, -1.5776e-01,  1.5796e-01,
         4.0752e-03, -1.7801e-01, -1.5906e-01, -4.3574e-03, -1.7708e-01,
         2.0009e-01,  1.5774e-01, -1.5778e-01, -1.7387e-01, -1.8227e-02,
         1.7813e-01,  1.8218e-02, -1.7175e-02, -1.8812e-02, -1.5895e-01,
         1.5572e-01, -1.5877e-01, -4.1667e-03,  1.5887e-01,  1.7361e-02,
         1.9610e-01,  1.6136e-03, -1.8031e-01,  1.9607e-02, -1.7379e-02,
        -1.5788e-01,  1.5771e-01, -1.5909e-01, -1.9657e-01,  1.5891e-01,
        -2.2286e-02, -1.5911e-01, -1.8511e-02, -1.8273e-02,  1.5909e-01,
        -1.9493e-01, -1.8406e-02,  1.5751e-01, -1.9616e-02, -1.9491e-01,
         2.9019e-01,  1.9659e-01, -1.5776e-01,  1.7377e-02, -1.5931e-01,
        -1.8756e-02,  1.7839e-01, -1.9658e-01,  1.8529e-02,  1.9802e-02,
        -1.9490e-01, -1.7711e-01,  1.9809e-02,  4.9

In [14]:
authorOf_relation = rel_to_id['<http://example.org/academic#authorOf>']
authorOf_relation_embedding = embeddings[cites_relation]
likely_author_embedding = likely_cited_paper_embedding - authorOf_relation_embedding 
#Since we have authorOf instead of hasAuthor, we use the reverse, otherwise we would have used addition
print(likely_author_embedding)

tensor([ 0.0799, -0.0785,  0.0979, -0.0979,  0.0784, -0.0979, -0.0800, -0.0967,
        -0.0800, -0.0966, -0.0968, -0.0979,  0.0785, -0.0784,  0.0786,  0.0797,
        -0.0798, -0.0799, -0.0799, -0.0786,  0.0981,  0.0784, -0.0784, -0.0981,
         0.0800,  0.0799, -0.0800, -0.0965, -0.0979, -0.0798,  0.0799, -0.0796,
        -0.0799,  0.0798,  0.0967,  0.0979, -0.0968, -0.0786, -0.0785, -0.0967,
        -0.0785,  0.0786, -0.0800, -0.0981,  0.0798,  0.0798, -0.0797, -0.0979,
         0.0799,  0.0800, -0.0967,  0.0798,  0.0784,  0.0786, -0.0967,  0.2144,
         0.0981, -0.0784,  0.0967, -0.0799, -0.0981,  0.0799, -0.0981, -0.0799,
        -0.0784, -0.0968, -0.0979, -0.0784, -0.0968, -0.0966, -0.0979, -0.0786,
        -0.0784, -0.0798,  0.0979, -0.0966,  0.0799, -0.0799, -0.0800,  0.0981,
         0.0799, -0.0966, -0.0965, -0.0799,  0.0786,  0.0799, -0.0967, -0.0797,
         0.0799, -0.0799,  0.0786,  0.0786,  0.0786, -0.0784,  0.0785, -0.0786,
        -0.0967, -0.0979, -0.0786, -0.09

In [17]:
def find_closest_author(mapping, likely_author_embedding):
    # Get all entities that are authors
    all_entities = list(mapping.keys())
    authors = [entity for entity in all_entities if 'Author' in entity]
    likely_author_embedding_np = likely_author_embedding.cpu().numpy()

    # Calculate distances
    distances = []
    for author in authors:
        author_embedding = embeddings[ent_to_id[author]]
        author_embedding_np = author_embedding.cpu().numpy()
        distance = euclidean(likely_author_embedding_np, author_embedding_np)
        distances.append((author, distance))

    # Find the author with the smallest distance
    closest_author = min(distances, key=lambda x: x[1])

    return closest_author[0], closest_author[1]

In [18]:
# Use the function
closest_author, distance = find_closest_author(ent_to_id, likely_author_embedding)

print(f"The closest author is: {closest_author}")
print(f"The Euclidean distance is: {distance}")

#The closest author is: <http://example.org/academic#Author_82277>
#The Euclidean distance is: 0.571506917476654

The closest author is: <http://example.org/academic#Author_82277>
The Euclidean distance is: 0.571506917476654
